This is the simplest bronze solution, using 9 stacked models for toxic classification. Unlucky we haven't submitted this solution.

In [ ]:
import gc
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import transformers
import optuna
from transformers import AdamW, AutoTokenizer, AutoModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import torch.nn.functional as F
import time
from tqdm import tqdm

import pickle
import warnings
warnings.simplefilter('ignore')

# Для подготовки данных
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import re

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!pip install --no-index --no-deps ../input/swifter/swifter-1.1.2.tar

In [ ]:
import swifter

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

In [ ]:
train = False

In [ ]:
def washing_machine(comments): # Чистка текста от мусора
    corpus=[]
    for i in tqdm(range(len(comments))):
        comment = re.sub('[^a-zA-Z]', ' ', comments[i])
        comment = comment.lower()
        comment = comment.split()
        stemmer = SnowballStemmer('english')
        lemmatizer = WordNetLemmatizer()
        all_stopwords = stopwords.words('english')
        comment = [stemmer.stem(word) for word in comment if not word in set(all_stopwords)]
        comment = [lemmatizer.lemmatize(word) for word in comment]
        comment = ' '.join(comment)
        corpus.append(comment)

    return corpus

# ✅ Data cleaning

In [ ]:
def clean(data, col):
    # чистка для tfidf

        # Remove ip address
    data[col] = data[col].str.replace(r'(([0-9]+\.){2,}[0-9]+)',' ')
    # Remove website
    data[col] = data[col].str.replace(r'https?://\S+|www\.\S+', ' ')
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3')
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1')
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1')
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1')
    data[col] = data[col].str.replace(r'[ ]{2,}',' ').str.strip()   
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\'"]+)',r' \1 ')    
    # Remove multiple white spaces
    data[col] = data[col].str.replace(r' +', ' ')
    # Remove html tags
    data[col] = data[col].str.replace(r'<[^<]+?>', ' ')
    
    #Code for removing slang words
    
    d = {'luv':'love','wud':'would','lyk':'like','wateva':'whatever','ttyl':'talk to you later',
         'kul':'cool','fyn':'fine','omg':'oh my god!','fam':'family','bruh':'brother',
         'cud':'could', 'fud':'food'} ## Need a huge dictionary
        
    data[col] = data[col].swifter.apply(lambda x : ' '.join(d[word] if word in d else word for word in x.split()))
    data[col] = data[col].str.replace(r"j k", "jk")
    data[col] = data[col].str.replace(r"e - mail", "email")
    data[col] = data[col].str.replace(r"\0s", "0")
    data[col] = data[col].str.replace(r" 9 11 ", " 911 ")
    data[col] = data[col].str.replace(r" e g ", " eg ")
    data[col] = data[col].str.replace(r" b g ", " bg ")
    data[col] = data[col].str.replace(r" u s ", " american ")
#         data[col] = data[col].str.replace(r"https?://[A-Za-z0-9./]+", "url")
    data[col] = data[col].str.replace(r"what's", "what is ")
    data[col] = data[col].str.replace(r"\'ve", " have ")
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
#     data[col] = data[col].str.replace(r"n ' t", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ")
    data[col] = data[col].str.replace(r"\'d", " would ")
    data[col] = data[col].str.replace(r"\'ll", " will ")
    data[col] = data[col].str.replace(r"\'scuse", " excuse ")
    data[col] = data[col].str.replace(r"\'s", " ")
    data[col] = data[col].str.replace('\n', ' \n ')
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)', r'\1 \2 \3')
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}', r'\1\1\1')
    data[col] = data[col].str.replace(r'([*!?\']+)', r' \1 ')
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b', r'\1\1')
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B', r'\1\1\1')
    data[col] = data[col].str.replace(r'[ ]{2,}', ' ').str.strip()
    data[col] = data[col].str.replace(r'[ ]{2,}', ' ').str.strip()

    return data

# df = clean(df, 'text')

# ✅ Model 1 - Roberta Base

In [ ]:
def get_predictions(args, dataloader, model):
    model.eval()
    all_outputs=[]
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    with torch.no_grad():
        for step, data in bar:
            batch_size = args.batch_size

            input_ids = data['input_ids'].to(device) # .cuda()
            attention_mask = data['attention_mask'].to(device) #.cuda()
            outputs = model(input_ids, attention_mask)
            outputs = outputs.cpu().detach().numpy()
            outputs = [sum(output) for output in outputs]
            all_outputs.append(outputs)

            bar.set_postfix(Stage='Inference') 
    return np.hstack(all_outputs)

In [ ]:
class Config:
    model_name = '../input/roberta-base'
    batch_size = 96
    lr = 1e-4
    weight_decay = 0.01
    scheduler = 'CosineAnnealingLR'
    early_stopping_epochs = 1
    epochs = 20
#     max_length = 128    
    max_length = 196
    num_folds = 2
    
    

class ToxicDataset:
    def __init__(self, comments, tokenizer, max_len=196):
        self.comments = comments
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.comments)
    
    def __getitem__(self, idx):
        
        tokenized = self.tokenizer.encode_plus(
            self.comments[idx],
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length'
        )
        
        input_ids = tokenized['input_ids']
        attention_mask = tokenized['attention_mask']
        

        return {
            'input_ids' : torch.tensor(input_ids, dtype=torch.long),
            'attention_mask' : torch.tensor(attention_mask, dtype=torch.long)
        }    
    
class ToxicModel(nn.Module):
    def __init__(self, args):
        super(ToxicModel, self).__init__()
        self.args = args
        self.model = AutoModel.from_pretrained(self.args.model_name)
        self.dropout = nn.Dropout(p=0.2)
        self.toxic = nn.Linear(768, 1)
        self.stoxic = nn.Linear(768, 1)
        self.obs = nn.Linear(768, 1)
        self.threat = nn.Linear(768, 1)
        self.insult = nn.Linear(768, 1)
        self.id_hate = nn.Linear(768, 1)
    
        
    def forward(self, input_ids, attention_mask):
        
        out = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=False
        )
        
        out = self.dropout(out[1])
        
        toxic = self.toxic(out)
        stoxic = self.stoxic(out)
        obs = self.obs(out)
        threat = self.threat(out)
        insult = self.insult(out)
        id_hate = self.id_hate(out)

        return torch.cat([toxic, stoxic, obs, threat, insult, id_hate], dim=-1)

In [ ]:
def inference(data):
    args=Config()
    print('Пытаюсь найти модель тут:',args.model_name)
    tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    base_path='../input/roberta-base-multi-label/'
    
    dataset = ToxicDataset(data, tokenizer)
    dataloader = DataLoader(dataset, batch_size=16*args.batch_size)
    
     
    
    final_preds = []
    
    num_folds = args.num_folds 
    
    for fold in range(num_folds):
        model = ToxicModel(args)
#         model = model.cuda()

        model.to(device)
        path = base_path + f'model_fold_{fold}.bin'
        model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
        
        print(f"Getting predictions for model {fold+1}")
        preds = get_predictions(args, dataloader, model)
        final_preds.append(np.vstack(preds))
        del model
        gc.collect()
    return np.hstack(sum(final_preds)/num_folds)

# Make Prediction

In [ ]:
if train:
    df_train = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")
    print(df_train.shape) 
    df_train.head()
    df_train['severe_toxic'] = df_train.severe_toxic * 3
    df_train['threat'] = df_train.threat * 2

    df_train['y'] = (df_train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) ).astype(int)
    df_train['y'] = np.around (df_train["y"].values , decimals = 2)

    df_train = df_train[['comment_text', 'y']].rename(columns={'comment_text': 'text'})

    # Reduce rows with 0 toxicity
    df_train = pd.concat([df_train[df_train.y>0] , 
                    df_train[df_train.y==0].sample(int(len(df_train[df_train.y>0])*1.5)) ], axis=0).sample(frac=1)

    print(df_train.shape)
    data_train = washing_machine(df_train['text'].values)
else:
    df = pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')
    data = washing_machine(df['text'].values)
    
    

In [ ]:
if train==True:
    df_train['score_M1'] = inference(data_train)
else:
    df['score_M1'] = inference(data)

# # df[['comment_id', 'score_M1']].to_csv('submission.csv', index=False)

# ✅ Model 2 - autonlp-toxic-new

In [ ]:
class ToxicDataset:
    def __init__(self, comments, tokenizer, max_len=196):
        self.comments = comments
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.comments)
    
    def __getitem__(self, idx):
        
        tokenized = self.tokenizer.encode_plus(
            self.comments[idx],
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length'
        )
        
        input_ids = tokenized['input_ids']
        attention_mask = tokenized['attention_mask']
        

        return {
            'input_ids' : torch.tensor(input_ids, dtype=torch.long),
            'attention_mask' : torch.tensor(attention_mask, dtype=torch.long)
        }    
    
    
def get_predictions(dataloader, model):
    
    model.to(device)
    model.eval()
    all_outputs=[]
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    with torch.no_grad():
        for step, data in bar:
            batch_size = 100

            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask)
            outputs = F.softmax(outputs.logits, dim = 1).cpu().detach().numpy()[:,1]
            all_outputs.append(outputs)

            bar.set_postfix(Stage='Inference') 
    return np.hstack(all_outputs)

In [ ]:
model_path = '../input/autonlp-toxic-1'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

if train==True:
    dataset_train = ToxicDataset(data_train, tokenizer)
    dataloader_train = DataLoader(dataset_train, batch_size=100)
    df_train['score_M2'] = get_predictions(dataloader_train, model)
else:
    dataset = ToxicDataset(data, tokenizer)
    dataloader = DataLoader(dataset, batch_size=100)
    df['score_M2'] = get_predictions(dataloader, model)
    

# ✅ Model 3 - distilbert-base-uncased-finetuned-sst-2-english

In [ ]:
def get_predictions(dataloader, model):
    model.to(device)
    model.eval()
    all_outputs=[]
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    with torch.no_grad():
        for step, data in bar:
            batch_size = 100
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask)
            outputs = F.softmax(outputs.logits, dim = 1).cpu().detach().numpy()[:,0]
            all_outputs.append(outputs)

            bar.set_postfix(Stage='Inference') 
    return np.hstack(all_outputs)

In [ ]:
model_path = '../input/distilbertbaseuncasedfinetunedsst2english/distilbert-base-uncased-finetuned-sst-2-english'

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForSequenceClassification.from_pretrained(model_path)

if train==True:
    dataset_train = ToxicDataset(data_train, tokenizer)
    dataloader_train = DataLoader(dataset_train, batch_size=100)
    df_train['score_M3'] = get_predictions(dataloader_train, model)
else:
    dataset = ToxicDataset(data, tokenizer)
    dataloader = DataLoader(dataset, batch_size=100)
    df['score_M3'] = get_predictions(dataloader, model)

# ✅ Model 4 - dehatebert-mono-english

In [ ]:
def get_predictions(dataloader, model):
    model.to(device)
    model.eval()
    all_outputs=[]
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    with torch.no_grad():
        for step, data in bar:
            batch_size = 100

            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask)
            outputs = F.softmax(outputs.logits, dim = 1).cpu().detach().numpy()[:,1]
            all_outputs.append(outputs)

            bar.set_postfix(Stage='Inference') 
    return np.hstack(all_outputs)

In [ ]:
model_path = '../input/hatespeechcnergdehatebertmonoenglish/dehatebert-mono-english'

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForSequenceClassification.from_pretrained(model_path)


if train==True:
    dataset_train = ToxicDataset(data_train, tokenizer)
    dataloader_train = DataLoader(dataset_train, batch_size=100)
    df_train['score_M4'] = get_predictions(dataloader_train, model)
else:
    dataset = ToxicDataset(data, tokenizer)
    dataloader = DataLoader(dataset, batch_size=100)
    df['score_M4'] = get_predictions(dataloader, model)

# ✅ Model 5 - twitter-roberta-base-hate

In [ ]:
def get_predictions(dataloader, model):
    model.to(device)
    model.eval()
    all_outputs=[]
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    with torch.no_grad():
        for step, data in bar:
            batch_size = 100
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask)
            outputs = F.softmax(outputs.logits, dim = 1).cpu().detach().numpy()[:,1]
            all_outputs.append(outputs)

            bar.set_postfix(Stage='Inference') 
    return np.hstack(all_outputs)

In [ ]:
model_path = '../input/twitterrobertabasehate/twitter-roberta-base-hate'

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForSequenceClassification.from_pretrained(model_path)


if train==True:
    dataset_train = ToxicDataset(data_train, tokenizer)
    dataloader_train = DataLoader(dataset_train, batch_size=100)
    df_train['score_M5'] = get_predictions(dataloader_train, model)
else:
    dataset = ToxicDataset(data, tokenizer)
    dataloader = DataLoader(dataset, batch_size=100)
    df['score_M5'] = get_predictions(dataloader, model)

# ✅ Model 6 - twitter-roberta-base-offensive

In [ ]:
model_path = '../input/twitterrobertabaseoffensive/twitter-roberta-base-offensive'


tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForSequenceClassification.from_pretrained(model_path)

if train==True:
    dataset_train = ToxicDataset(data_train, tokenizer)
    dataloader_train = DataLoader(dataset_train, batch_size=100)
    df_train['score_M6'] = get_predictions(dataloader_train, model)
else:
    dataset = ToxicDataset(data, tokenizer)
    dataloader = DataLoader(dataset, batch_size=100)
    df['score_M6'] = get_predictions(dataloader, model)

# ✅ Model 7 - twitter-roberta-base-sentiment

In [ ]:
def get_predictions(dataloader, model):
    model.to(device)
    model.eval()
    all_outputs=[]
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    with torch.no_grad():
        for step, data in bar:
            batch_size = 100
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask)
            outputs = F.softmax(outputs.logits, dim = 1).cpu().detach().numpy()[:,0]
            all_outputs.append(outputs)

            bar.set_postfix(Stage='Inference') 
    return np.hstack(all_outputs)

In [ ]:
model_path = '../input/twitterrobertabasesentiment/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

if train==True:
    dataset_train = ToxicDataset(data_train, tokenizer)
    dataloader_train = DataLoader(dataset_train, batch_size=100)
    df_train['score_M7'] = get_predictions(dataloader_train, model)
else:
    dataset = ToxicDataset(data, tokenizer)
    dataloader = DataLoader(dataset, batch_size=100)
    df['score_M7'] = get_predictions(dataloader, model)

# ✅ Model 8 - BERT-SBIC-offensive

In [ ]:
def get_predictions(dataloader, model):
    model.to(device)
    model.eval()
    all_outputs=[]
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    with torch.no_grad():
        for step, data in bar:
            batch_size = 100
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask)
            outputs = F.softmax(outputs.logits, dim = 1).cpu().detach().numpy()[:,2]
            all_outputs.append(outputs)

            bar.set_postfix(Stage='Inference') 
    return np.hstack(all_outputs)

In [ ]:
model_path = '../input/bertsbicoffensive/BERT-SBIC-offensive'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

if train==True:
    dataset_train = ToxicDataset(data_train, tokenizer)
    dataloader_train = DataLoader(dataset_train, batch_size=100)
    df_train['score_M8'] = get_predictions(dataloader_train, model)
else:
    dataset = ToxicDataset(data, tokenizer)
    dataloader = DataLoader(dataset, batch_size=100)
    df['score_M8'] = get_predictions(dataloader, model)

# ✅ Model 9 - beep-KcELECTRA-base-hate (3 classes)

In [ ]:
def get_predictions(dataloader, model):
    model.to(device)
    model.eval()
    all_outputs=[]
    all_outputs_features = {'none':[], 
                            'offensive': [],
                            'hate': []
                           }
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    with torch.no_grad():
        for step, data in bar:
            batch_size = 100
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask)
            outputs = F.softmax(outputs.logits, dim = 1).cpu().detach().numpy()#[:,[1,2]].sum(axis=1)
            all_outputs.append(outputs[:,[1,2]].sum(axis=1))
            all_outputs_features['none'].append(outputs[:,0])
            all_outputs_features['offensive'].append(outputs[:,1])
            all_outputs_features['hate'].append(outputs[:,2])
            bar.set_postfix(Stage='Inference') 
            
    all_outputs_features['none'] = np.hstack(all_outputs_features['none'])
    all_outputs_features['offensive'] = np.hstack(all_outputs_features['offensive'])
    all_outputs_features['hate'] = np.hstack(all_outputs_features['hate'])
            
    return np.hstack(all_outputs), pd.DataFrame(all_outputs_features)

In [ ]:
model_path = '../input/beepkcelectrabasehate/beep-KcELECTRA-base-hate'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

if train==True:
    dataset_train = ToxicDataset(data_train, tokenizer)
    dataloader_train = DataLoader(dataset_train, batch_size=100)
    df_train['score_M9'], df_feat_train = get_predictions(dataloader_train, model)
    df_feat_train.sample(3)
else:
    dataset = ToxicDataset(data, tokenizer)
    dataloader = DataLoader(dataset, batch_size=100)
    df['score_M9'], df_feat = get_predictions(dataloader, model)
    df_feat.sample(3)

# ✅ Model 10 - bertweet-hate-speech(3 classes)

In [ ]:
# def get_predictions(dataloader, model):
#     model.to(device)
#     model.eval()
#     all_outputs=[]
#     all_outputs_features = {'hateful':[], 
#                             'targeted': [],
#                             'aggresive': []
#                            }
#     bar = tqdm(enumerate(dataloader), total=len(dataloader))
#     with torch.no_grad():
#         for step, data in bar:
#             batch_size = 100
#             input_ids = data['input_ids'].to(device)
#             attention_mask = data['attention_mask'].to(device)
#             outputs = model(input_ids, attention_mask)
#             outputs = F.softmax(outputs.logits, dim = 1).cpu().detach().numpy()#[:,[0,2]].sum(axis=1)
#             all_outputs.append(outputs)#[:,[0,2]].sum(axis=1))
#             all_outputs_features['hateful'].append(outputs[:,0])
#             all_outputs_features['targeted'].append(outputs[:,1])
#             all_outputs_features['aggresive'].append(outputs[:,2])
#             bar.set_postfix(Stage='Inference') 
            
#     all_outputs_features['hateful'] = np.hstack(all_outputs_features['hateful'])
#     all_outputs_features['targeted'] = np.hstack(all_outputs_features['targeted'])
#     all_outputs_features['aggresive'] = np.hstack(all_outputs_features['aggresive'])
            
#     return np.hstack(all_outputs), pd.DataFrame(all_outputs_features)

In [ ]:
# model_path = '../input/bertweethatespeech/bertweet-hate-speech'


# tokenizer = AutoTokenizer.from_pretrained(model_path)

# model = AutoModelForSequenceClassification.from_pretrained(model_path)

# dataset = ToxicDataset(data, tokenizer, max_len=130)
# dataloader = DataLoader(dataset, batch_size=100)


# df['score_M10'],df_feat1 = get_predictions(dataloader, model)

# dataset_train = ToxicDataset(data_train, tokenizer)
# dataloader_train = DataLoader(dataset_train, batch_size=100)
# df_train['score_M10'], df_feat_train1 = get_predictions(dataloader_train, model)

In [ ]:
if train==True:
    df_train_full = pd.concat([df_train.reset_index(drop=True),df_feat_train],axis = 1)
    print(df_train_full.columns)
    df_train_full.sample(5)
if train==False:
    df_full = pd.concat([df.reset_index(drop=True),df_feat],axis = 1)
    print(df_full.columns)
    df_full.sample(5)

# ❎ Ridge regression

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from mlxtend.feature_selection import ColumnSelector

import sklearn
sklearn.set_config(display='diagram') # Визуализация пайплайна


In [ ]:
if train==True:
    df_train_full['text'] = clean(df_train,'text')['text'].values
else:
    df_full['text'] = clean(df,'text')['text'].values
    


In [ ]:
text_features = ['text']
bert_features = ['score_M1', 'score_M2', 'score_M3', 'score_M4',
                 'score_M5', 'score_M6', 'score_M7', 'score_M8',
                 'score_M9', 'none', 'offensive', 'hate']

In [ ]:
text_transformer = Pipeline(steps=[
    ('col_selector', ColumnSelector(cols=('text'),drop_axis=True)),
    ("tfidf", TfidfVectorizer(
        min_df = 5, max_df = 0.7, 
        norm = 'l2', analyzer = 'char_wb', ngram_range = (3, 5))),
])

#   Для численных/бертовых - применяем SimpleImputer,
bert_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer()),    
    
])

# Собираем воедино трансформеры для текстовых и бертовых признаков
data_transformer = ColumnTransformer(transformers=[
    ("text_fs", text_transformer, text_features),
    ("bert_fs", bert_transformer, bert_features),
])

# Создание конвейера препроцессора, который сначала преобразует данные
preprocessor = Pipeline(steps=[("data_transformer", data_transformer)])

ridge_pipline = Pipeline(
    steps=[("preprocessor", preprocessor), 
           ("regressor", Ridge(alpha=1.0))])

ridge_pipline

In [ ]:
# Для сериализации моделей
import pickle

In [ ]:
if train==True:
    ridge_pipline.fit(df_train_full.drop('y', axis=1), df_train_full['y'])

    with open("owesome_pipeline.pkl", "wb") as f:
        pickle.dump(ridge_pipline, f)
else:
    with open("../input/d/dzisandy/jigsaw-ridge-reg-v1/owesome_pipeline.pkl", 'rb') as f:
        ridge_pipline = pickle.load(f)

In [ ]:
ridge_pipline['regressor'].coef_

In [ ]:
if train==False:
    df['score_rd_big_2'] = ridge_pipline.predict(df_full)
#     df[['comment_id','score']].to_csv('submission.csv', index=False)


# ✅✅✅ 3 Alerons Piplines with Ridge

In [ ]:
def clean(data, col):
    
    def remove_urls (vTEXT):
        vTEXT = re.sub(r'http\S+|www\S+', 'URL', vTEXT)
        return(vTEXT)

    data[col] = data[col].swifter.swifter.apply(remove_urls)
    
    data[col] = data[col].str.replace(r"-", " - ", regex=True)
    
    #Removing numbers
    data[col] = data[col].swifter.swifter.apply(lambda x : ' '.join([tweet for tweet in x.split() if not tweet == '\d*']))
    
        # Remove ip address
    data[col] = data[col].str.replace(r'(([0-9]+\.){2,}[0-9]+)',' ', regex=True)
    # Remove website
    data[col] = data[col].str.replace(r'https?://\S+|www\.\S+', ' ', regex=True)
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3', regex=True)
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1', regex=True)
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1', regex=True)
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1', regex=True)
    data[col] = data[col].str.replace(r'[ ]{2,}',' ', regex=True).str.strip()   
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\'"]+)',r' \1 ', regex=True)    
    # Remove multiple white spaces
    data[col] = data[col].str.replace(r' +', ' ', regex=True)
    # Remove html tags
    data[col] = data[col].str.replace(r'<[^<]+?>', ' ', regex=True)
    
    #Code for removing slang words
    
    d = {'luv':'love','wud':'would','lyk':'like','wateva':'whatever','ttyl':'talk to you later',
         'kul':'cool','fyn':'fine','omg':'oh my god!','fam':'family','bruh':'brother',
         'cud':'could', 'fud':'food'} ## Need a huge dictionary
        
    data[col] = data[col].swifter.swifter.apply(lambda x : ' '.join(d[word] if word in d else word for word in x.split()))
    data[col] = data[col].str.replace(r"j k", "jk", regex=True)
    data[col] = data[col].str.replace(r"e - mail", "email", regex=True)
    data[col] = data[col].str.replace(r"\0s", "0", regex=True)
    data[col] = data[col].str.replace(r" 9 11 ", " 911 ", regex=True)
    data[col] = data[col].str.replace(r" e g ", " eg ", regex=True)
    data[col] = data[col].str.replace(r" b g ", " bg ", regex=True)
    data[col] = data[col].str.replace(r" u s ", " american ", regex=True)
    data[col] = data[col].str.replace(r"what's", "what is ", regex=True)
    data[col] = data[col].str.replace(r"\'ve", " have ", regex=True)
    data[col] = data[col].str.replace(r"can't", "cannot ", regex=True)
    data[col] = data[col].str.replace(r"n't", " not ", regex=True)
    data[col] = data[col].str.replace(r"i'm", "i am ", regex=True)
    data[col] = data[col].str.replace(r"\'re", " are ", regex=True)
    data[col] = data[col].str.replace(r"\'d", " would ", regex=True)
    data[col] = data[col].str.replace(r"\'ll", " will ", regex=True)
    data[col] = data[col].str.replace(r"\'scuse", " excuse ", regex=True)
    data[col] = data[col].str.replace(r"\'s", " ", regex=True)
    data[col] = data[col].str.replace('\n', ' \n ', regex=True)
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)', r'\1 \2 \3', regex=True)
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}', r'\1\1\1', regex=True)
    data[col] = data[col].str.replace(r'([*!?\']+)', r' \1 ', regex=True)
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b', r'\1\1', regex=True)
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B', r'\1\1\1', regex=True)
    data[col] = data[col].str.replace(r'[ ]{2,}', ' ', regex=True).str.strip()
    data[col] = data[col].str.replace(r'[ ]{2,}', ' ', regex=True).str.strip()

    def count_urls(text):
        return text.count('URL')
    
    data['URL_count'] = data[col].swifter.swifter.apply(count_urls)

#     data[col] = data[col].swifter.swifter.apply(lambda x: ' '.join([word for word in x.split() if word not in (filtered_stop_word)]))
    return data

In [ ]:
from string import punctuation

def up_low(s):
    u, l  = sum(1 for i in s if i.isupper()), sum(1 for i in s if i.islower())
    if l == 0 : l = 1
    return u/l

def up_low_word(s):
    u, l  = sum(1 for i in s.split() if i.isupper()), sum(1 for i in s.split() if i.islower())
    if l == 0 : l = 1
    return u/l



# расстояние левинштейна


def get_toxic_index(text, words = ['fuck', 'bitch', None]):
    sen_list = sent_tokenize(text)
    sen_list = [sen.lower() for sen in sen_list]
    one_word_sum = 0
    sent_words = len(text.split())
    text = text.lower()
    for word in words:
        one_word_sum += text.count(word)/sent_words*np.log(1 + (len(sen_list)+1)/(sum(1 for sen in sen_list if sen.count(word))+1))
#         one_word_sum += sum(sen.lower().count(word)/len(sen.split()) for sen in sen_list)*np.log(1 + (len(sen_list)+1)/(sum(1 for sen in sen_list if sen.lower().count(word))+1))
    return one_word_sum


def make_features(df, target = 'text', show_fs=False):
    
    col_before = df.columns
    
    df['sentence_count'] = df[target].swifter.apply(lambda x : len(sent_tokenize(x)))
    df['upper_on_lower'] = df[target].swifter.apply(up_low)
    df['upper_on_lower_word'] = df[target].swifter.apply(up_low_word)
    df['len_text'] = df[target].swifter.apply(lambda x : len(x)//30)
    df['size_punct'] = df[target].swifter.apply(lambda x : sum(1 for i in x if i in punctuation)) # Надо сколько именно слов, не символов
    df['size_letter'] = df[target].swifter.apply(lambda x : sum(1 for i in x if i not in punctuation))
    df['punct_on_letter'] = df['size_punct'].values/df['size_letter'].values
    df.drop(['size_letter', 'size_punct'], axis=1, inplace=True)
#     df['mean_word_len'] = df[target].swifter.apply(lambda x : sum(len(i)//3 for i in x.split())/len(x.split()))
    df['words'] = df[target].swifter.apply(lambda x : sum(1 for i in x.split())/len(x))
    df['ques_punct_on_word'] = df[target].swifter.apply(lambda x : 100*sum(1 for i in x if i in '?')/len(x))
    df['state_punct_on_word'] = df[target].swifter.apply(lambda x : 100*sum(1 for i in x if i in '!')/len(x))
    df['duble_state_punct_on_word'] = df[target].swifter.apply(lambda x : 100*sum(1 for i in x if i in '!!')/len(x))
    df['snow_punct_on_word'] = df[target].swifter.apply(lambda x : 100*sum(1 for i in x if i in '*')/len(x))
    df['uniq_word_on_sentence'] = df[target].swifter.apply(lambda x : len(set(x.split()))/len(x.split()))*df['sentence_count'].values
    df['haha_pattern'] = df[target].swifter.apply(lambda x : x.lower().count('hah')/len(x.split()))*df['sentence_count'].values
    df['fuck_pattern'] = df[target].swifter.apply(lambda x : x.lower().count('fuck')/len(x.split()))*df['sentence_count'].values
    df['off_pattern'] = df[target].swifter.apply(lambda x : x.lower().count('off')/len(x.split()))*df['sentence_count'].values
    df['ques_pattern'] = df[target].swifter.apply(lambda x : x.count('???')/len(x.split()))*df['sentence_count'].values
    df['you_pattern'] = df[target].swifter.apply(lambda x : x.lower().count(' you')/len(x.split()))*df['sentence_count'].values
    ################################
    
    # Правильная нормировка 
#     df['sentence_count']

#     df[target] = df[target].swifter.apply(get_toxic_index, words = words)
    ###############################
    # TODO ur
    df['u_pattern'] = df[target].swifter.apply(lambda x : len(x.lower().split(' u '))/len(x.split()))
    
    you_are_patterns = ['you are', 'you ar ', 'you re ', 'u are ', ' u r ', ' are you', ' ur ', "you ' re", "you're", 'youre',
                        'you will', "you'll", "you ' ll", 'u will'] 
    
#     df['you_are_patterns'] = df[target].swifter.apply(lambda x : sum(len(x.split(i)) for i in you_are_patterns )/len(x.split()))*df['sentence_count'].values
    df['you_are_patterns'] = df[target].swifter.apply(get_toxic_index, words = you_are_patterns)
    
    
    hide_punct_pattern = '*&%$#@'
#     df['hide_punct_pattern'] = df[target].swifter.apply(lambda x : sum(x.count(i) for i in hide_punct_pattern )/len(x.split()))*df['sentence_count'].values
    df['hide_punct_pattern'] = df[target].swifter.apply(get_toxic_index, words = hide_punct_pattern)

    penis_pattern = ['dick','diick', 'penis', 'sock','suck', 'cock', 'balls', ' ball', 'bell', 'phallus','condone',
                     'sucker', 'suker', 'penes', 'prick', 'pecker', 'dildo']
#     df['penis_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in penis_pattern)/len(x.split()))*df['sentence_count'].values
    df['penis_pattern'] = df[target].swifter.apply(get_toxic_index, words = penis_pattern)
    
    fucking_pattern = ['fuck', '*uck', 'f*ck', 'fu*k', 'fuc*','f**k','f ** k', 'fucck','fcuken','fcken','ruck', 'f *** ing', 'sh * t', 'sh*t','s**t', 'fukk', 'fricki',
                       'f***', 'f ** ck', 'f * ck', 'b ** ch','b * ch']
#     df['fucking_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in fucking_pattern)/len(x.split()))*df['sentence_count'].values  
    df['fucking_pattern'] = df[target].swifter.apply(get_toxic_index, words = fucking_pattern)
    
    virgin_pattern = ['virgin', 'slut', 'whore', 'prostitut', 'bitch', 'biatch', 'hooker','pussy', 'beetch', 'bithc', 'butch']
#     df['virgin_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in virgin_pattern)/len(x.split()))*df['sentence_count'].values 
    df['virgin_pattern'] = df[target].swifter.apply(get_toxic_index, words = virgin_pattern)
    
    dubble_word_pattern = ['buttseck', 'dumbass', 'asshole', 'arsehole', 'cocksucker', 'asscake', 'jackass']
#     df['dubble_word_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in dubble_word_pattern)/len(x.split()))*df['sentence_count'].values
    df['dubble_word_pattern'] = df[target].swifter.apply(get_toxic_index, words = dubble_word_pattern)
    
    funny_pattern = ['glad', 'happy', 'hilarious', 'fanny', 'humor']
#     df['funny_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in funny_pattern)/len(x.split()))*df['sentence_count'].values
    df['funny_pattern'] = df[target].swifter.apply(get_toxic_index, words = funny_pattern)
    
   # ДОбавить поправку растояния левенштена                                                                       
    toxic_words = ['ass', 'a$$', 'hole', 'drool', 'anal', 'pedo', 'poopie', 'hose', 'berk', 'duffer', 'pillock', 'plonker', 'wally',
                   'knobend', 'rotter', 'swine', 'blighter', ' cad', 'idiot', 'faggot', 'clown', 'tosser', 'cunt', 'arrogant', 'freak', 'dude',  
                   'ugly', 'little', 'fat', 'liar', 'heck', 'vandal', 'nerd','stubborn']
#     df['toxic_word_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in toxic_words)/len(x.split()))*df['sentence_count'].values 
    df['toxic_word_pattern'] = df[target].swifter.apply(get_toxic_index, words = toxic_words)    
        
    yeald_pattern = ['ggg','uuu','aaa', 'hhh', 'eee','ooo','!!!','???', '!?!', '!??']
#     df['yeald_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in yeald_pattern)/len(x.split()))*df['sentence_count'].values    
    df['yeald_pattern'] = df[target].swifter.apply(get_toxic_index, words = yeald_pattern)    
        
    noncence_toxic = ['poo', 'poop', 'poopy', 'butt','piss','shit','shut', 'testicles', 'bollocks', 'heresy', 'absurd', 'nonsense', 'crud','crap']
#     df['noncence_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in noncence_toxic)/len(x.split()))*df['sentence_count'].values
    df['noncence_toxic'] = df[target].swifter.apply(get_toxic_index, words = noncence_toxic)
    
    long_phase_pattern = ['fuck all', 'piss off', 'fuck off', 'get off', 'stay off', 'shut up', 'god damn', 'damn it', 'who care', 'kiss off', 'fuck–up', 'fuck up', 'bang up']
#     df['long_phase_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in long_phase_pattern)/len(x.split()))*df['sentence_count'].values
    df['long_phase_pattern'] = df[target].swifter.apply(get_toxic_index, words = long_phase_pattern)

    child_toxic = ['holy', 'moly', 'jeepers', 'good', 'heavens', 'gosh', 'fiddlesticks', 'poppycock']
#     df['child_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in child_toxic)/len(x.split()))*df['sentence_count'].values
    df['child_toxic'] = df[target].swifter.apply(get_toxic_index, words = child_toxic)

    more_toxic_word = ['damn', 'whore', 'hoe', 'jade', 'jerk', 'moron', 
                       'douchebag', 'dork', 'faggot', 'fag', 'queer', 'pirate', 'candyass',
                       'finook', 'capon', 'butter', 'boy', 'banana', 'crammer', 'nancy', 'pansy', 'scumbag', 'scum', 'prat', 'brat', 'loser',
                       'sack', 'goof', 'wanker', 'noob', 'retard', 'numbnuts', 'shitbox']
#     df['more_toxic_word'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in more_toxic_word)/len(x.split()))*df['sentence_count'].values
    df['more_toxic_word'] = df[target].swifter.apply(get_toxic_index, words = more_toxic_word)
    
    light_level_toxic  = ['arse', 'bloody', 'hell', 'bugger', 'damn', 'minger', 'sod-off']
#     df['light_level_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in light_level_toxic)/len(x.split()))*df['sentence_count'].values
    df['light_level_toxic'] = df[target].swifter.apply(get_toxic_index, words = light_level_toxic)
    
    middle_level_toxic = ['shit', 'son of a bitch', 'arsehole', 'balls', 'bint', 'bollocks', 'bullshit', 'feck', 'munter', 'pissed', 'pissed off']
#     df['middle_level_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in middle_level_toxic)/len(x.split()))*df['sentence_count'].values
    df['middle_level_toxic'] = df[target].swifter.apply(get_toxic_index, words = middle_level_toxic)
    
    hard_level_toxic  = ['bastard', 'dickhead', 'bloodclaat', 'jamaican', 'knob', 'prick', 'bellend',
                        'minge', 'twat', 'twunt', 'beaver', 'curtains', 'beef', 'clunge', 'snatch', 'punani', 'gash']
#     df['hard_level_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in hard_level_toxic)/len(x.split()))*df['sentence_count'].values
    df['hard_level_toxic'] = df[target].swifter.apply(get_toxic_index, words = hard_level_toxic)
    
    very_hard_level_toxic  = ['cunt', 'fuck', 'motherfucker', 'motherfudger', 'mother fudger']
#     df['very_hard_level_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in very_hard_level_toxic)/len(x.split()))*df['sentence_count'].values
    df['very_hard_level_toxic'] = df[target].swifter.apply(get_toxic_index, words = very_hard_level_toxic)
    
    very_very_hard_level_toxic  = ['felch' ,'skullfuck', 'fuck puppet', 'bisnotch', 'rusty trombone']
#     df['very_very_hard_level_toxic'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in very_very_hard_level_toxic)/len(x.split()))*df['sentence_count'].values
    df['very_very_hard_level_toxic'] = df[target].swifter.apply(get_toxic_index, words = very_very_hard_level_toxic)
    
    away_patterns  = [' out ', 'away ', ' off ' ,' up ', ' down ', 'm done']
#     df['away_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in away_patterns)/len(x.split()))*df['sentence_count'].values
    df['away_patterns'] = df[target].swifter.apply(get_toxic_index, words = away_patterns) 
        
    synon_toxic_words = ['jerk', 'stinker', 'heck', 'crud', 'schmuck', 'riffraff']    
#     df['synon_toxic_words'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in synon_toxic_words)/len(x.split()))*df['sentence_count'].values
    df['synon_toxic_words'] = df[target].swifter.apply(get_toxic_index, words = synon_toxic_words)
    
    lgbt_pattern = [ 'gay', 'homosexual', 'hetero', ' lgbt', 'bisexual', 'lesbian', 'queer']
#     df['lgbt_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in lgbt_pattern)/len(x.split()))*df['sentence_count'].values
    df['lgbt_pattern'] = df[target].swifter.apply(get_toxic_index, words = lgbt_pattern)
    
    descrimination_pattern = ['racist', 'nazi', 'antic', 'commie',  'discriminat', 'female', 'troll', 'abusing', 'nigger', 'nigga', 'jew',
                            'fascist', 'haras', 'racism', 'bigot', 'gosh', 'kike', 'slave', 'victim', ]
#     df['descrimination_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in descrimination_pattern)/len(x.split()))*df['sentence_count'].values
    df['descrimination_pattern'] = df[target].swifter.apply(get_toxic_index, words = descrimination_pattern)
    
    god_pattern = ['catholic', 'god','religi', 'islam', 'bibl', 'muslim', 'atheist']
#     df['god_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in god_pattern)/len(x.split()))*df['sentence_count'].values
    df['god_pattern'] = df[target].swifter.apply(get_toxic_index, words = god_pattern)
    
    animal_pattern = ['monkey', 'horse', 'pig', 'animal', 'dog', 'bull', ' swine', ' pork', 'llama','skunk','racooon', 'raccoon', 'racoon', ' coon',
                    'bat', 'bird', 'chicken', 'duck', 'goose', 'weasel', ' swan', 'dog', 'donkey', 'goat', ' rat', 'sheep']
#     df['animal_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in animal_pattern)/len(x.split()))*df['sentence_count'].values
    df['animal_pattern'] = df[target].swifter.apply(get_toxic_index, words = animal_pattern)
    
    slang_pattern = [' u ', ' ur ', 'cuz']
#     df['slang_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in slang_pattern)/len(x.split()))*df['sentence_count'].values
    df['slang_pattern'] = df[target].swifter.apply(get_toxic_index, words = slang_pattern)
    
    body_pattern = ['tits', 'mouth', 'body', 'face',  'head', ' boob', 'oral']  
#     df['body_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in body_pattern)/len(x.split()))*df['sentence_count'].values
    df['body_pattern'] = df[target].swifter.apply(get_toxic_index, words = body_pattern)
    
    not_good_words = ['rude', 'stuck', 'porn', 'mug', 'sex', 'shame', 'gang', 'bang', 'wanker']  
#     df['not_good_words'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in not_good_words)/len(x.split()))*df['sentence_count'].values
    df['not_good_words'] = df[target].swifter.apply(get_toxic_index, words = not_good_words)
    
    threat_pattern = ['going', 'wanna', 'will', 'threat', 'barbarian', 'savage', 'insult', 'rape',]
#     df['threat_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in threat_pattern)/len(x.split()))*df['sentence_count'].values
    df['threat_pattern'] = df[target].swifter.apply(get_toxic_index, words = threat_pattern)
    
    hard_threat_words = ['xenocidic', 'terrorist', 'akbar', 'dangerous', 'genocide', 'holocaust']
#     df['hard_threat_words'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in hard_threat_words)/len(x.split()))*df['sentence_count'].values
    df['hard_threat_words'] = df[target].swifter.apply(get_toxic_index, words = hard_threat_words)
    
    stop_pattern = ['delet', 'destroy', ' bann', 'block',  'stop', 'quit', 'unblock', 'attack', 'remove']
#     df['stop_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in stop_pattern)/len(x.split()))*df['sentence_count'].values
    df['stop_pattern'] = df[target].swifter.apply(get_toxic_index, words = stop_pattern)
    
    mom_pattern = [' mom ', 'mother', ' moth ', ' mum ', 'mothefucker', ' milf', ]  
#     df['mom_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in mom_pattern)/len(x.split()))*df['sentence_count'].values
    df['mom_pattern'] = df[target].swifter.apply(get_toxic_index, words = mom_pattern)
    
    brother_pattern = [' son ', 'son of', 'children', 'cousin', 'kids', 'baby', 'brother', 'father','family', 'daugher', 'sister']  
#     df['brother_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in brother_pattern)/len(x.split()))*df['sentence_count'].values
    df['brother_pattern'] = df[target].swifter.apply(get_toxic_index, words = brother_pattern)
    
    mental_pattern = ['retard', 'mind', 'mental', 'brain', 'stupid', 'silly', 'fool', 'dumb', 'meatball', 'numnut',
                      'paranoid', 'mad', 'crazy', 'insane', 'frantic', 'reckless', 'lunatic', 'sick']  
#     df['mental_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in mental_pattern)/len(x.split()))*df['sentence_count'].values
    df['mental_pattern'] = df[target].swifter.apply(get_toxic_index, words = mental_pattern)
    
    negative_pattern = ['horrible', 'coward', 'crack','less','dart','hate', 'annoy', 'hurt', 'demon',
                        'pain', 'dead', ' die',' death', 'satan', 'terrible', 'awful', 'spam', 'avil']
#     df['negative_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in negative_pattern)/len(x.split()))*df['sentence_count'].values   
    df['negative_pattern'] = df[target].swifter.apply(get_toxic_index, words = negative_pattern)
    
    negative_verb_pattern = ['smell', 'stink', ' reek', ' hum']
#     df['negative_verb_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in negative_verb_pattern)/len(x.split()))*df['sentence_count'].values    
    df['negative_verb_pattern'] = df[target].swifter.apply(get_toxic_index, words = negative_verb_pattern)
    
    ill_pattern = [ 'canser', 'coward', 'disease', 'slave', 'bizarre', 'anorexic', ' cry', 'obese', 'fart','garbage']
#     df['ill_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in ill_pattern)/len(x.split()))*df['sentence_count'].values
    df['ill_pattern'] = df[target].swifter.apply(get_toxic_index, words = ill_pattern)
    
    crime_pattern = ['warmonger', 'pedophile', 'kill', 'murder', 'vandal', 'crime', 'fraud', 'scam', 'gulty', 'speculat']
#     df['crime_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in crime_pattern)/len(x.split()))*df['sentence_count'].values
    df['crime_pattern'] = df[target].swifter.apply(get_toxic_index, words = crime_pattern)
    
    nasty_pattern = ['nasty', 'bad ', 'black', 'filth', 'horrible','horror','suicid', 'masturb', 'disgust', 'slime', 'slur']
#     df['nasty_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in nasty_pattern)/len(x.split()))*df['sentence_count'].values
    df['nasty_pattern'] = df[target].swifter.apply(get_toxic_index, words = nasty_pattern)
    
    talkative_pattern = ['talker', 'chatterbox', 'windbag', 'babbler', 'gossip', 'jay'] 
#     df['talkative_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in talkative_pattern)/len(x.split()))*df['sentence_count'].values
    df['talkative_pattern'] = df[target].swifter.apply(get_toxic_index, words = talkative_pattern)
    
    target_pattern = ['self', 'selves'] 
#     df['target_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in target_pattern)/len(x.split()))*df['sentence_count'].values
    df['target_pattern'] = df[target].swifter.apply(get_toxic_index, words = target_pattern)
    
    WTF_pattern = ['what the', 'wtf', 'omfg', 'why the'] 
#     df['WTF_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in WTF_pattern)/len(x.split()))*df['sentence_count'].values
    df['WTF_pattern'] = df[target].swifter.apply(get_toxic_index, words = WTF_pattern)
    
    country_pattern = ['moldavian', 'iraq', 'egypt','romanian','countr', 'israel', 'russia', 'europa', 'america', 'livin',  'arab','chinese','czech',
                       'china', 'spain', 'germanic', 'germany', 'poland','texas', 'russkie', 'france', 'armenia', 'rebublic', 'macedonia', 'bulgar',
                       'albania',' turk', 'london','greek', 'natioa', 'population', 'india', 'hindu', 'turk', 'saudi', 'pakistan'] 
#     df['country_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in country_pattern)/len(x.split()))*df['sentence_count'].values
    df['country_pattern'] = df[target].swifter.apply(get_toxic_index, words = country_pattern)
    
    politics_pattern = ['putin', 'bush', 'abama', 'trump', 'stalin', 'kgb', 'saddam', 'hussein', 'politic', 'president', 'cult',
                        'anarchist', 'osama bin', 'hitler', ' war', ' fbi', 'lenin', 'biden', 'liberal', 'propagand']  
#     df['politics_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in politics_pattern)/len(x.split()))*df['sentence_count'].values
    df['politics_pattern'] = df[target].swifter.apply(get_toxic_index, words = politics_pattern)
    
    NO_pattern = [' no', 'nooo', 'not','nor', 'none', 'no one', 'nobody', 'nothing', 'neither', 'nowhere', 'never', 'negative', ' un-', 'anti', 'pseudo'] 
#     df['NO_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in NO_pattern)/len(x.split()))*df['sentence_count'].values
    df['NO_pattern'] = df[target].swifter.apply(get_toxic_index, words = NO_pattern)
    
    NO_verb_pattern = ["do't", "do ' t", 'doesn’t', 'isn’t', 'wasn’t', 'shouldn’t', 'wouldn’t', "couldn’t", "can't", "can ' t", "cant"] 
#     df['NO_verb_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in NO_verb_pattern)/len(x.split()))*df['sentence_count'].values
    df['NO_verb_pattern'] = df[target].swifter.apply(get_toxic_index, words = NO_verb_pattern)
    
    max_pattern = ["all!", 'every', 'any', 'too ', 'such', 'very'] 
#     df['max_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in max_pattern)/len(x.split()))*df['sentence_count'].values
    df['max_pattern'] = df[target].swifter.apply(get_toxic_index, words = max_pattern)
    
    thanks_pattern = ["sorry", " thank", 'thx', 'hello', 'hey']  
#     df['thanks_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in thanks_pattern)/len(x.split()))*df['sentence_count'].values
    df['thanks_pattern'] = df[target].swifter.apply(get_toxic_index, words = thanks_pattern)
    
    positive_pattern = ["well", "nice", 'good', 'cool'] 
#     df['positive_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in positive_pattern)/len(x.split()))*df['sentence_count'].values
    df['positive_pattern'] = df[target].swifter.apply(get_toxic_index, words = positive_pattern)
    
    weapon_pattern = ["gun", "weapon", 'rocket', 'shoot', 'bomb'] 
#     df['weapon_pattern'] = df[target].swifter.apply(lambda x : sum(x.lower().count(word) for word in weapon_pattern)/len(x.split()))*df['sentence_count'].values
    df['weapon_pattern'] = df[target].swifter.apply(get_toxic_index, words = weapon_pattern)

    new_features = [i for i in df.columns if i not in col_before]
    
    if show_fs: print('new_features', len(new_features), new_features)
    
    return df, new_features

In [ ]:
with open("../input/big-ridge-piplines/pipeline.pkl", 'rb') as f:
    ridge_pipline_1 = pickle.load(f)
    
with open("../input/big-ridge-piplines/ridge_pipeline_1.pkl", 'rb') as f:
    ridge_pipline_2 = pickle.load(f)
    
with open("../input/big-ridge-piplines/ridge_pipeline_2.pkl", 'rb') as f:
    ridge_pipline_3 = pickle.load(f)

In [ ]:
df_test = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")


df_test, _ = make_features(df_test, target = 'text')
df_test = clean(df_test, 'text')
coms_id = df_test.comment_id.values
df_test.drop('comment_id', axis=1, inplace=True)

m1_preds = ridge_pipline_1.predict(df_test)

df['score_M11'] = rankdata(m1_preds, method='ordinal') # m1_preds
# df_test['comment_id'] = coms_id


# with open("pipeline.pkl", "wb") as f:
#     pickle.dump(pipeline, f)


In [ ]:
jr_preds=ridge_pipeline_2.predict(df_test)
df['score_M12']=rankdata(jr_preds, method='ordinal') 

with open("ridge_pipeline_1.pkl", "wb") as f:
    pickle.dump(ridge_pipeline_1, f)


In [ ]:
jr_preds=ridge_pipeline_3.predict(df_test)
df['score_M13']=rankdata(jr_preds, method='ordinal') 

with open("ridge_pipeline_1.pkl", "wb") as f:
    pickle.dump(ridge_pipeline_1, f)


# Final Final blend two big Ridges


In [ ]:
if train==False:
    df['score_rd_big_2'] = ridge_pipline.predict(df_full)
#     df[['comment_id','score']].to_csv('submission.csv', index=False)

    


In [ ]:
# if train==False:
df['score'] = df['score_rd_big_2'].values + df['score_M11'].values + df['score_M12'].values + df['score_M13'].values
df[['comment_id','score']].to_csv('submission.csv', index=False)

    


# ❎ Блендинг

In [ ]:
# weights = {#'score_M1': 1,
#           'score_M2': 1,
#           'score_M3': 1,
#           'score_M4': 1,
#           'score_M5': 1,
#           'score_M6': 1,
#           'score_M7': 1,
#           'score_M8': 1,
# #           'score_M9': 1
#           }

In [ ]:
#Подозрительно, что среденее по score_M1 не ноль. =>  Убрал пока из прогноза 
# df[['score_M1','score_M2','score_M3','score_M4','score_M5','score_M6','score_M7', 'score_M8', 'score_M9']].mean(axis=0)

In [ ]:
# for mod, weight in weights.items():
#     df[mod] = df[mod].rank()*weight
    
# df['score'] = df[weights.keys()].sum(axis=1).rank(method='max')

In [ ]:
# df[['comment_id','score']].to_csv('submission.csv', index=False)
# df[['comment_id','score']].sample(10)